In [4]:
import tensorflow as tf
import gym
import numpy as np
from collections import deque
import time
import tqdm
import pybullet_envs

In [8]:
class DDPG():
    
    def __init__(self, env, gamma = 0.9, replay_buffer_size = 1000, learning_rate = 0.0003):
        self._env = env
        self._gamma = gamma
        self._learning_rate = learning_rate
        self._replay_buffer_size = replay_buffer_size
        self._graph = {}
        self._obs_dim = self._env.observation_space.shape[0]
        self._n_acts = self._env.action_space.shape[0]
        self._evaluation_scores = []
        self._build_computational_graph_continuous_actions()

    def train(self, n_iterations):
        pass
    
    def _build_computational_graph_continuous_actions(self):
        cg = self._graph
        self._define_replay_buffer()
        self._define_placeholder()
        self._define_networks()
        
        action_means = cg['actor_network'](cg['obs_ph'])
        tfd = tfp.distributions
        dist = tfd.Normal(loc=action_means, scale=np.ones(self._n_acts))
        cg['actions'] = dist.sample(1)[0]
        
        action_means_target = cg['actor_target'](cg['new_obs_ph'])
        critic_input =  tf.concat([new_obs_ph, action_means_target], axis = 1)
        y = rew_ph + self._gamma * cg['critic_target'](critic_input) * (1-terminal_ph)
        old_action_values = cg['critic_network'](tf.concat([obs_ph, act_ph], axis = 1))
        action_value_loss = tf.reduce_mean((tf.stop_gradient(y)-old_action_values)**2)
        # todo: define policy loss
        
        self._define_optimizer()
        self._sess = tf.Session()
        self._sess.run(tf.global_variables_initializer())
    
    def _define_replay_buffer(self):
        memory_width = 3 + 2*self._obs_dim
        self._experience_replay_buffer = np.zeros((self._replay_buffer_size, memory_width)),
        self._row_pointer = 0
        
    def _define_placeholder(self):
        self._graph['obs_ph'] = tf.placeholder(shape=(None, self._obs_dim), dtype=tf.float32)
        self._graph['act_ph'] = tf.placeholder(shape=(None,), dtype=tf.int32)
        self._graph['new_obs_ph'] = tf.placeholder(shape=(None, self._obs_dim), dtype=tf.float32)
        self._graph['rew_ph'] = tf.placeholder(shape=(None,1), dtype=tf.float32)
        self._graph['terminal_ph'] = tf.placeholder(shape=(None,1), dtype=tf.float32)

    def _define_networks(self):
        self._graph['critic_network'] = self._build_dense_model()
        self._graph['critic_target'] = self._build_dense_model()
        self._graph['actor_network'] = self._build_dense_model(self._n_acts, 'tanh')
        self._graph['actor_target'] = self._build_dense_model(self._n_acts, 'tanh')
        
    def _build_dense_model(self, n_outputs = 1, activation = 'relu'):
        mlp = tf.keras.models.Sequential()
        mlp.add(tf.keras.layers.Dense(50, activation=activation))
        mlp.add(tf.keras.layers.Dense(50, activation=activation))
        mlp.add(tf.keras.layers.Dense(n_outputs, activation=None))
        return mlp
    
    def _define_optimizer(self):
        policy_optimizer = tf.train.AdamOptimizer(self._learning_rate)
        self._graph['train_policy'] = policy_optimizer.minimize(self._graph['policy_loss'])
        action_value_optimizer = tf.train.AdamOptimizer(self._learning_rate)
        self._graph['train_action_value'] = action_value_optimizer.minimize(self._graph['action_value_loss'])

In [5]:
env = gym.make('HalfCheetahBulletEnv-v0')
model = DDPG(env)
model.train(100)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WalkerBase::__init__ start
WARN: Environment '<class 'pybullet_envs.gym_locomotion_envs.HalfCheetahBulletEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


/home/janus/repos/rl-zoo/venv/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
